In [1]:
import matplotlib.pyplot as plt
import numpy as np
import egg.core as core
import pandas as pd

from utils.analysis_tools_lazimpa import *
from utils.analysis_from_interaction import *
from archs import Sender, Receiver
from train import loss
from models_lazimpa import LazImpaSenderReceiverRnnGS

In [2]:
dataset = '(3,4)'
values = 4
attributes = 3
n_epochs = 300
path = 'results/' + dataset + '_game_size_10_vsf_3'
context_unaware = False # whether original or context_unaware simulations are evaluated
lazy = True # whether the lazy agent is used
setting = ""
if not lazy:
    if context_unaware:
        setting = 'context_unaware'
    else:
        setting = 'standard'
else:
    if context_unaware:
        setting = 'lazy_context_unaware'
    else:
        setting = 'lazy_context_aware'
run = 0

In [151]:
# Start what kind of variables do I need for attribute visualization

interaction = load_interaction(path,setting,run,n_epochs)
concepts, context_condition = retrieve_concepts_context(interaction,values)
messages = retrieve_messages(interaction,False)
message_length = MessageLengthHierarchical.compute_message_length(messages)
unique_messages, frequencies = torch.unique(messages,dim=0,return_counts=True)
unique_length = MessageLengthHierarchical.compute_message_length(unique_messages)

m = [f"{ *m, }" for m in messages.numpy()] # as str for hashing
l = message_length.numpy()
cx = context_condition
fint = [list(map(int,list(f))) for c,f in concepts]
fix = [f"{v}" for v in fint]
c = [list( map(int, list( [a if fa == 1. else -1 for a,fa in zip(c[0],f) ] ))) for c,f in concepts]
um = [f"{ *m, }" for m in unique_messages.numpy()]
ul = unique_length.numpy()
f = frequencies.numpy()

results/(3,4)_game_size_10_vsf_3/lazy_context_aware/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([1807, 20, 12])


In [158]:
# TODO Make all symbols after eos also eos: Are there any? Why in qualitative but not here?

In [161]:
# make one DataFrame containing all the data we have
in_list = []
for i in range(message_length.shape[0]):
    in_list.append((fix[i],fint[i], np.where(concepts[i][1]==1.0)[0].tolist(), sum(fint[i]), c[i], cx[i], m[i], l[i])) # np.where(cc[i][1]==1.0)[0].tolist()[0]
df = pd.DataFrame(in_list,columns=["fixed", "fixed int","which fixed indices", "nr attributes", "concept example", "context condition", "message", "message length"])
df

,fixed,fixed int,which fixed indices,nr attributes,concept example,context condition,message,message length
0,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 3, 1]",1,"(4, 0, 0, 0)",1
1,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 0, 3]",1,"(11, 0, 0, 0)",1
2,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 1, 3]",1,"(5, 0, 0, 0)",1
3,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[1, 3, 3]",2,"(10, 5, 0, 0)",2
4,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 1]",2,"(4, 0, 0, 0)",1
...,...,...,...,...,...,...,...,...
1802,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 1, 0]",1,"(2, 12, 0, 0)",2
1803,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[0, 3, 3]",2,"(6, 9, 0, 0)",2
1804,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[3, -1, 2]",0,"(7, 0, 0, 0)",1
1805,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 2, 1]",0,"(8, 0, 0, 0)",1


In [165]:
# calculate additional data that can be merged
unique_df = pd.DataFrame([(m,f) for m,f in zip(list(um),list(f))],columns=["message", "usage frequency"])
total_data_df = df.merge(unique_df,how='right',on="message")
total_data_df

,fixed,fixed int,which fixed indices,nr attributes,concept example,context condition,message,message length,usage frequency
0,"[1, 1, 1]","[1, 1, 1]","[0, 1, 2]",3,"[2, 2, 3]",1,"(1, 0, 0, 0)",1,52
1,"[1, 0, 1]","[1, 0, 1]","[0, 2]",2,"[0, -1, 3]",0,"(1, 0, 0, 0)",1,52
2,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 2, 3]",1,"(1, 0, 0, 0)",1,52
3,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 2, 3]",0,"(1, 0, 0, 0)",1,52
4,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 2, 3]",1,"(1, 0, 0, 0)",1,52
...,...,...,...,...,...,...,...,...,...
1802,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 0, -1]",0,"(14, 14, 0, 0)",2,8
1803,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 0, -1]",0,"(14, 14, 0, 0)",2,8
1804,"[0, 1, 0]","[0, 1, 0]",[1],1,"[-1, 0, -1]",0,"(14, 14, 0, 0)",2,8
1805,"[0, 1, 1]","[0, 1, 1]","[1, 2]",2,"[-1, 0, 1]",0,"(14, 14, 0, 0)",2,8


# Data Analysis

In [5]:
message_length.float().mean() # weighted by occurence

tensor(1.5368)

In [6]:
# How long are the messages on average used for a certain fix attribute number
scores = message_length_per_hierarchy_level(interaction, attributes)
scores # These values are correct!

message length tensor([1, 1, 1,  ..., 1, 1, 2])
hierarchical [1.607, 1.544, 1.53]


[1.607, 1.544, 1.53]

In [162]:
# How long are the messages on average used in the different context conditions
grouped_by_context = df.groupby(["context condition"])['message length'].apply(lambda l: l.mean()).reset_index()
grouped_by_context

,context condition,message length
0,0,1.281333
1,1,1.601757
2,2,1.930481


In [169]:
# how long are the messages on average for different numbers of fixed attributes
grouped_by_nr_fixed_attributes = df.groupby(["nr attributes","context condition"])["message length"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes

,nr attributes,context condition,message length
0,1,0,1.607143
1,2,0,1.363036
2,2,1,1.729730
3,3,0,1.171355
4,3,1,1.503876
5,3,2,1.930481


In [ ]:
# how long are the messages on average for different attributes
grouped_by_nr_fixed_attributes = df.groupby(["which attributes","context condition"])["message length"].apply(lambda l: l.mean()).reset_index()
grouped_by_nr_fixed_attributes

In [168]:
# how often are messages used with different lengths in total
#grouped_by_length = pd.DataFrame([(l,f) for l,f in zip(ul,f)],columns=["length","frequency"])
grouped_by_length = total_data_df.groupby(["message length"])['usage frequency'].apply(lambda f: (f.mean(),f.std(ddof=False))).reset_index()
grouped_by_length[['mean usage frequency', 'std usage frequency']] = pd.DataFrame(grouped_by_length['usage frequency'].tolist(), index=grouped_by_length.index)
grouped_by_length = grouped_by_length.drop('usage frequency',axis=1)
grouped_by_length

,message length,mean usage frequency,std usage frequency
0,1,71.648045,26.524830
1,2,17.269321,11.625401
2,3,6.034483,2.994049


In [27]:
# how long are messages used for a certain number and position of fixed attributes (only in fine context) # used means all the messages that are used also if they are the same used twice
context = 1 # 1 means 2 (fine), 2 means 1, 0 means -3 (coarse) für 3 fixed attributes
in_list = []
for i in range(message_length.shape[0]):
    #if cc[i][1].sum() == 1: # all that have exactly 1 fixed attribute
    if cx[i] + context == cc[i][1].sum(): # Only the ones with specific context
        content = [int(sl) if sl == 1. else "" for sl in list(cc[i][1])]
        content.append(l[i])
        in_list.append(content) # np.where(cc[i][1]==1.0)[0].tolist()[0]
df = pd.DataFrame(in_list,columns=["fixed 1", "fixed 2", "fixed 3", "av message length"])
df = df.groupby(['fixed 1','fixed 2','fixed 3'])['av message length'].apply(lambda l: l.mean()).reset_index()
df

,fixed 1,fixed 2,fixed 3,av message length
0,1,1,1,1.930481
1,1,1,,1.846939
2,1,,1,1.755102
3,1,,,1.600000
4,,1,1,1.590000
5,,1,,1.619048
6,,,1,1.600000
